In [1]:
!pip3 install pystan prophet

In [7]:
import pandas as pd
from prophet import Prophet

In [3]:
df = pd.read_csv('dataset.csv')

In [4]:
print(df['Product'].unique())
print(df['Store'].unique())

[2667437]
['QLD_CW_ST0203']


In [5]:
df['Year'] = df['Time Date'].apply(lambda x: str(x)[-4:])
df['Month'] = df['Time Date'].apply(lambda x: str(x)[-6:-4])
df['Day'] = df['Time Date'].apply(lambda x: str(x)[:-6])
df['ds'] = pd.DatetimeIndex(df['Year'] + '-' + df['Month'] + '-' + df['Day'])
df.dtypes

Time Date             int64
Product               int64
Store                object
Value               float64
Year                 object
Month                object
Day                  object
ds           datetime64[ns]
dtype: object

In [6]:
df.drop(['Time Date', 'Product', 'Store', 'Year', 'Month', 'Day'], axis=1, inplace=True)
df.columns = ['y', 'ds']

## Training

- `interval_width=0.95` -> handles trend uncertainty (as opposed to uncertainty in seasonality)
    - by allowing higher flexibility rate, `changepoint_prior_scale`, we also increase he forecast uncertainty
    - This is because if we model more rate changes in the history then we will expect more in the future, and makes the uncertainty intervals a useful indicator of overfitting.

In [11]:

m = Prophet(interval_width=0.95)
m.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -33.1653
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      53       1945.53   0.000631867       102.449   3.995e-06       0.001      101  LS failed, Hessian reset 
      99       1946.09   0.000368073       98.8744      0.6795      0.6795      159   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     108       1946.15   0.000160306       71.7704   1.585e-06       0.001      207  LS failed, Hessian reset 
     199       1946.31   0.000475004       40.7529      0.5331      0.9399      319   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     200       1946.31   7.99931e-05       58.5875   1.963e-06       0.001      387  LS failed, Hessian reset 
     272       1946.44   1.04944e-05       89.2853   1.249e-07       0.001      523  LS failed, Hessian reset 
     299       1946.44   7.84896e-08       58.5545      0.3264   